## 导入数据

In [26]:
import sklearn.feature_extraction
import pandas as pd
import os
import csv
import numpy as np

这里只有一个正常sql的文件名为legit，剩余文件数据为sqli语句，正常的标0，其余标1

In [13]:
basedir = 'data'
filelist = os.listdir(basedir) 
df_list = []
for file in filelist:
    #osos.path.join(basedir,file)的作用是将参数转化成“basedir/file”的路径
    df = pd.read_csv(os.path.join(basedir,file), quoting=csv.QUOTE_NONE, names=['raw_sql'], header=None)
    df['label'] = 0 if file.split('.')[0] == 'legit' else 1
    df_list.append(df)
dataframe = pd.concat(df_list, ignore_index=True)
dataframe.dropna(inplace=True)
print (dataframe['label'].value_counts())
dataframe.head()

1    12672
0     1003
Name: label, dtype: int64


,raw_sql,label
0,) AND 93=93,1
1,) AND 9170=2793,1
2,') AND 7364=5114,1
3,') AND 93=93,1
4,') AND 4731=891,1


把sql标记化

In [14]:
import sqlparse
def parse_it(raw_sql):
    parsed = sqlparse.parse(str(raw_sql))
    return [token._get_repr_name() for parse in parsed for token in parse.tokens if token._get_repr_name() != 'Whitespace']

dataframe['parsed_sql'] = dataframe['raw_sql'].map(lambda x: parse_it(x))

In [48]:
dataframe.head()

,raw_sql,label,parsed_sql
0,) AND 93=93,1,"[Punctuation, Keyword, Comparison]"
1,) AND 9170=2793,1,"[Punctuation, Keyword, Comparison]"
2,') AND 7364=5114,1,"[Error, Punctuation, Keyword, Comparison]"
3,') AND 93=93,1,"[Error, Punctuation, Keyword, Comparison]"
4,') AND 4731=891,1,"[Error, Punctuation, Keyword, Comparison]"
5,' AND 5874=2869,1,"[Error, Keyword, Comparison]"
6,' AND 93=93,1,"[Error, Keyword, Comparison]"
7,' AND 529=7832,1,"[Error, Keyword, Comparison]"
8,""" AND 5183=1335",1,"[Error, Keyword, Comparison]"
9,""" AND 93=93",1,"[Error, Keyword, Comparison]"


使用ngram来展现各标记之间的联系

In [7]:
def ngrams(lst):
    N = 3
    ngrams = []
    for n in range(0,N):
        # *表示转置
        ngrams += zip(*(lst[i:] for i in range(n+1)))
    return [str(tuple) for tuple in ngrams]

In [25]:
matrix = dataframe.as_matrix()
X = matrix[:, 0]
Y = matrix[:, 1]
#改变label的数据类型，便于后面的预测
Y = np.array(Y, dtype=np.int)

/Users/dqy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

数据向量化

In [18]:
vectorizer = TfidfVectorizer(tokenizer=ngrams)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

## 训练数据

In [19]:
clf = RandomForestClassifier()
#一般训练数据使用fit_transform来转换
X_train = vectorizer.fit_transform(X_train)
clf.fit(X_train, y_train)
X_test = vectorizer.transform(X_test)
y_probs= clf.predict_proba(X_test)[:,1]

/Users/dqy/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## 打印准确率和召回率

In [22]:
print("accuracy:%f"%(sklearn.metrics.accuracy_score(y_test, y_probs>.9)))
print("recall:%f"%(sklearn.metrics.recall_score(y_test, y_probs>=.9)))

accuracy:0.991959
recall:0.995290


## 测试

In [42]:
def test_it(sql):
    parsed_sql = parse_it(sql)
    _X = vectorizer.transform(parsed_sql)
    my=clf.predict(_X)[0]
    if(my):
        print("%s is a SQL injection statement"%sql)
    else:
        print("%s is benign"%sql)

In [43]:
test_it('select * from employees')
test_it("'; exec master..xp_cmdshell")
test_it("'any 'x'='x'")
test_it('from dorseys mom xp_cmdshell biache')
test_it('select * from your_mom')

select * from employees is benign
'; exec master..xp_cmdshell is a SQL injection statement
'any 'x'='x' is a SQL injection statement
from dorseys mom xp_cmdshell biache is a SQL injection statement
select * from your_mom is benign
